Para usar este script es necesario configurar las siguientes environment variables:
- `AWS_ACCESS_KEY`
- `AWS_SECRET_ACCESS_KEY`
- `AWS_DEFAULT_REGION`

Cargando dependencias y conectando al recurso dynamoDB

In [142]:
import boto3
from boto3.dynamodb.conditions import Attr, Key
import pandas as pd
import json

dynamodb= boto3.resource('dynamodb')

# Creando Tabla 'Ordenes'

In [5]:
ordenes = dynamodb.create_table(
    TableName = 'Ordenes',
    KeySchema =[
        {
            'AttributeName' : 'OrdenID',
            'KeyType' : 'HASH'
        }
    ],
    AttributeDefinitions = [
        {
            'AttributeName' : 'OrdenID',
            'AttributeType' : 'S'
        }
    ],
    ProvisionedThroughput = {
        'ReadCapacityUnits' : 10,
        'WriteCapacityUnits' : 10
    }
)

ordenes.wait_until_exists()

print(ordenes.item_count)

0


Agregando un Item

In [ ]:
ordenes = dynamodb.Table('Ordenes')

ordenes.put_item(
    Item = {
        'OrdenID' : "189440",
        'Cliente' : [
            509477760, "Hertha Jiles",  "hjiles@ed.gov",     "Hombre",      "New Jersey",           False,                  30
        #   ClienteID   #Nombre          #Email                #Genero            #Lugar      #Internacional o no     #Edad
        ],
        'Producto' : [
            "BF8238",        1,              4
        #   ProductID       #Cantidad       #Rating
        ],
        "Precio" : [
            100,                0,              100
        #   PrecioSales        #Shipping        #Total
        ],
        'Reseña' :'The delivery team handled the product with care.'
    }
)

Agregando varios items en batch

In [92]:
with ordenes.batch_writer() as batch:
    batch.put_item(
        Item = {
            'OrdenID' : "126695",
            'Cliente' : [
                396566981, "Hertha Jiles",  "hjiles@ed.gov",     "Hombre",      "Sydney",           True,                  27
            #   ClienteID   #Nombre          #Email                #Genero            #Lugar      #Internacional o no     #Edad
            ],
            'Producto' : [
                "BF9708",        1,              5
            #   ProductID       #Cantidad       #Rating
            ],
            "Precio" : [
                97,                100,              197
            #   PrecioSales        #Shipping        #Total
            ],
            'Reseña' :'The Product exhibits good design.'
        }
    )

    batch.put_item(
        Item = {
            'OrdenID' : "129765",
            'Cliente' : [
                396566981, "Hertha Jiles",  "hjiles@ed.gov",     "Mujer",      "San Antonio",           False,                  19
            #   ClienteID   #Nombre          #Email                #Genero            #Lugar      #Internacional o no     #Edad
            ],
            'Producto' : [
                "BF6604",        1,              5
            #   ProductID       #Cantidad       #Rating
            ],
            "Precio" : [
                9,                0,              9
            #   PrecioSales        #Shipping        #Total
            ],
            'Reseña' :'Exquisite craftsmanship here is absolutely appreciated.'
        }
    )

# Queries

In [130]:
response = ordenes.get_item(
    Key = {
        'OrderID' : "189440"
    }
)

item = response['Item']
print(item)

{'ProductID': 'BF8238', 'Review': 'The delivery team handled the product with care.', 'ClientID': '509477760', 'BuyerAge': Decimal('30'), 'OrderDate': '21/07/2024', 'ShippingCharges': Decimal('0'), 'Quantity': Decimal('1'), 'BuyerGender': 'Male', 'InternationalShipping': 'No', 'OrderID': '189440', 'SalesPrice': Decimal('100'), 'TotalSales': Decimal('100'), 'SalesPerUnit': Decimal('100'), 'OrderLocation': 'New Jersey', 'Rating': Decimal('4')}


In [61]:
ordenes = dynamodb.Table('Ordenes')
response = ordenes.scan(
    ProjectionExpression = 'Cliente[1], OrdenID, Producto[0]',
    FilterExpression = Attr('Cliente').contains(396566981)
)

item = response['Items']


for i in range(len(item)):
    print(item[i]['Producto'])


['BF6167']
['BF1681']


# Creando tabla 'Productos'

In [10]:
productos = dynamodb.create_table(
    TableName = 'Productos',
    KeySchema =[
        {
            'AttributeName' : 'ProductID',
            'KeyType' : 'HASH'
        }
    ],
    AttributeDefinitions = [
        {
            'AttributeName' : 'ProductID',
            'AttributeType' : 'S'
        }
    ],
    ProvisionedThroughput = {
        'ReadCapacityUnits' : 10,
        'WriteCapacityUnits' : 10
    }
)

productos.wait_until_exists()

print(productos.item_count)

0


In [13]:
with productos.batch_writer() as batch:
    batch.put_item(
        Item = {
            'ProductID' : "BF9708",
            'Producto' : "Bufanda1",
            'Categoria' : "Accesorio",
            'Descripcion' :'Bufanda uno',
            'imgURL' : "https://productosizzyshop.s3.us-east-1.amazonaws.com/bufanda1.JPG"
        }
    )

    batch.put_item(
        Item = {
            'ProductID' : "Bf1414",
            'Producto' : "Camisa1",
            'Categoria' : "Prenda",
            'Descripcion' :'Camisa uno',
            'imgURL' : "https://productosizzyshop.s3.us-east-1.amazonaws.com/camisa1.JPG"
        }
    )

    batch.put_item(
        Item = {
            'ProductID' : "BF6604",
            'Producto' : "Collar1",
            'Categoria' : "Accesorio",
            'Descripcion' :'Collar uno',
            'imgURL' : "https://productosizzyshop.s3.us-east-1.amazonaws.com/collar1.JPG"
        }
    )

    batch.put_item(
        Item = {
            'ProductID' : "BF2887",
            'Producto' : "Zapatos1",
            'Categoria' : "Accesorio",
            'Descripcion' :'Zapato uno',
            'imgURL' : "https://productosizzyshop.s3.us-east-1.amazonaws.com/zapatos1.JPG"
        }
    )

    batch.put_item(
        Item = {
            'ProductID' : "BF8063",
            'Producto' : "Vestido1",
            'Categoria' : "Prenda",
            'Descripcion' :'Vestido uno',
            'imgURL' : "https://productosizzyshop.s3.us-east-1.amazonaws.com/vestido1.JPG"
        }
    )

In [157]:
from decimal import Decimal

class DecimalEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, Decimal):
      return float(obj)
    return json.JSONEncoder.default(self, obj)

productos =dynamodb.Table('productos')
response = productos.scan(
    #ProjectionExpression = 'Producto, Categoria, descripcion, urls3',

    FilterExpression = Attr('ProductID').eq('BF0809')
)

items = response['Items'][0]

print(json.dumps(items, cls=DecimalEncoder))

{"ProductID": "BF0809", "descripcion": "zapatos con una combinacion de colores perfecta y comodos para el dia a dia.", "Producto": "Zapatos3", "categoria": "Accesorio", "urls3": "https://productosizzyshop.s3.us-east-1.amazonaws.com/zapatos3.JPG", "precio": 45.99}


# Query en las dos tablas

In [166]:
productos =dynamodb.Table('productos')
ordenes = dynamodb.Table('Ordenes')
clienteID = "896030670"

cliResponse = ordenes.scan(
    ProjectionExpression = 'NameClient',
    FilterExpression = Attr('ClientID').contains(clienteID)
)
cliNom = cliResponse['Items'][0]
print(cliNom)

{'NameClient': 'Reggie Elijah'}


In [177]:
response = ordenes.scan(
    ProjectionExpression = 'OrderID, ProductID',
    FilterExpression = Attr('ClientID').eq(clienteID)
)

item = response['Items']
print(item)
for i in range(len(item)):
    productID = str(item[i]['ProductID'])

    response = productos.query(
        #ProjectionExpression = 'Producto, Categoria, Descripcion, imgURL',
        KeyConditionExpression = Key('ProductID').eq(productID)
    )
    print("response")

    # print(productID, "\t-", response['Items'])

    item[i]['ProductoDetalles'] = response['Items'][0]

item.insert(0, cliNom)

queryResults = item
print(queryResults)

[{'ProductID': 'BF8063', 'OrderID': '177684'}, {'ProductID': 'BF1772', 'OrderID': '168714'}]
response
response
[{'NameClient': 'Reggie Elijah'}, {'ProductID': 'BF8063', 'OrderID': '177684', 'ProductoDetalles': {'ProductID': 'BF8063', 'descripcion': 'Vestido rojo con puntos blancos con un estilo clasico pero juvenil a su vez.', 'Producto': 'Vestido1', 'categoria': 'Prenda', 'urls3': 'https://productosizzyshop.s3.us-east-1.amazonaws.com/vestido1.JPG', 'precio': Decimal('51.99')}}, {'ProductID': 'BF1772', 'OrderID': '168714', 'ProductoDetalles': {'ProductID': 'BF1772', 'descripcion': 'jeans de color gris oscuro, doblados. Los jeans tienen un bolsillo trasero visible y una etiqueta en la pretina.', 'Producto': 'Pantalones2', 'categoria': 'Prenda', 'urls3': 'https://productosizzyshop.s3.us-east-1.amazonaws.com/panatalon2.JPG', 'precio': Decimal('41')}}]


In [115]:
cliente =396566981
cliResponse = ordenes.scan(
    ProjectionExpression = 'Cliente[1]',
    FilterExpression = Attr('Cliente').contains(cliente)
)
cliNom = cliResponse['Items'][0]
print(cliNom)

response = ordenes.scan(
    ProjectionExpression = 'OrdenID, Producto[0]',
    FilterExpression = Attr('Cliente').contains(cliente)
)

item = response['Items']

for i in range(len(item)):
    productID = str(item[i]['Producto'][0])

    response = productos.query(
        #ProjectionExpression = 'Producto, Categoria, Descripcion, imgURL',
        KeyConditionExpression = Key('ProductID').eq(productID)
    )

    item[i]['ProductoDetalles'] = response['Items'][0]

item.insert(0, cliNom)
queryResults = item
print(queryResults)

{'Cliente': ['Hertha Jiles']}
[{'Cliente': ['Hertha Jiles']}, {'OrdenID': '129765', 'Producto': ['BF6604'], 'ProductoDetalles': {'ProductID': 'BF6604', 'Categoria': 'Accesorio', 'Producto': 'Collar1', 'Descripcion': 'Collar uno', 'imgURL': 'https://productosizzyshop.s3.us-east-1.amazonaws.com/collar1.JPG'}}, {'OrdenID': '126695', 'Producto': ['BF9708'], 'ProductoDetalles': {'ProductID': 'BF9708', 'Categoria': 'Accesorio', 'Producto': 'Bufanda1', 'Descripcion': 'Bufanda uno', 'imgURL': 'https://productosizzyshop.s3.us-east-1.amazonaws.com/bufanda1.JPG'}}]
